In [ ]:
import numpy as np
import numpy.linalg
import pandas as pd
import sklearn
import sklearn.preprocessing, sklearn.cluster, sklearn.metrics, sklearn.decomposition
import scipy.spatial
import matplotlib.pyplot as plt
import seaborn as sns

# Dimensionality Reduction and Matrix Factorization

## Objectives:
1. Apply SVD (complete) and Reduced SVD method to Bars Dataset and evaluate also data with noise.
2. Reconstruct the data with different reduction (2, 5, 10, 16 basis functions).
3. Try the NNMF method similarly.
4. Take the Mnist dataset and apply SVD recomposition and visualize it in vector space. Coloring take from Labels
5. Try to reconstruct the reduced data and visualize it.
6. Use PCA and t-SNE for further visualization of the Mnist dataset

---


## Exercise 1: Bars Dataset - SVD Analysis

**Task:** Apply SVD decomposition to the bars dataset.

- Apply complete SVD decomposition to noisy and clean data
- Visualize the singular values and compare clean vs noisy data
- Analyze the differences in singular value patterns


## Exercise 2: Data Reconstruction with Different Reduction Levels

**Task:** Reconstruct the Bars Dataset using different numbers of basis functions.

- Test reconstruction with 2, 5, 10, and 16 basis functions
- Calculate reconstruction error (MSE) for each reduction level
- Visualize original vs reconstructed data
- Plot reconstruction error vs number of components



## Exercise 3: Non-Negative Matrix Factorization (NNMF)

**Task:** Apply NNMF to the Bars Dataset and compare with SVD results.

- Implement NNMF decomposition for the Bars Dataset
- Apply NNMF with different numbers of components (2, 5, 10, 16)
- Compare NNMF reconstruction quality with SVD
- Visualize NNMF components and reconstructions
- Analyze reconstruction errors for both methods


## Exercise 4: MNIST Dataset - SVD Analysis and Visualization

**Task:** Apply SVD decomposition to MNIST dataset and visualize in vector space.

- Apply SVD decomposition to MNIST data
- Analyze singular values and cumulative explained variance
- Visualize MNIST data in 2D using first two SVD components
- Color points by digit labels
- Try different component combinations (e.g., components 2 and 3)


## Exercise 5: PCA and t-SNE Visualization of MNIST Dataset

**Task:** Apply PCA and t-SNE for dimensionality reduction and visualization of MNIST.

- Apply PCA to MNIST dataset
- Visualize MNIST data in 2D PCA space
- Apply t-SNE for non-linear dimensionality reduction
- Compare SVD, PCA, and t-SNE visualizations
- Color all visualizations by digit labels

